In [2]:
import pandas as pd
import os
import pymongo
from pymongo import MongoClient
from google.cloud import bigquery

def dataframeToMongo(data, collectionName):
    client = MongoClient('localhost',int(27017))
    db = client["DataVizMgmt"]
    collection = db[collectionName]
    my_list = data.to_dict('records')
    l = len(my_list)
    ran = range(l)
    steps=ran[100::100]
    steps.extend([l])
    i = 0
    for j in steps:
        collection.insert_many(my_list[i:j]) # fill de collection
        i = j

class BigQueryHelper(object):
    """
    Helper class to simplify common BigQuery tasks like executing queries,
    showing table schemas, etc without worrying about table or dataset pointers.

    See the BigQuery docs for details of the steps this class lets you skip:
    https://googlecloudplatform.github.io/google-cloud-python/latest/bigquery/reference.html
    """

    BYTES_PER_GB = 2**30

    def __init__(self, active_project, dataset_name, max_wait_seconds=180):
        self.project_name = active_project
        self.dataset_name = dataset_name
        self.max_wait_seconds = max_wait_seconds
        self.client = bigquery.Client()
        self.__dataset_ref = self.client.dataset(self.dataset_name, project=self.project_name)
        self.dataset = None
        self.tables = dict()  # {table name (str): table object}
        self.__table_refs = dict()  # {table name (str): table reference}
        self.total_gb_used_net_cache = 0

    def __fetch_dataset(self):
        # Lazy loading of dataset. For example,
        # if the user only calls `self.query_to_pandas` then the
        # dataset never has to be fetched.
        if self.dataset is None:
            self.dataset = self.client.get_dataset(self.__dataset_ref)

    def __fetch_table(self, table_name):
        # Lazy loading of table
        self.__fetch_dataset()
        if table_name not in self.__table_refs:
            self.__table_refs[table_name] = self.dataset.table(table_name)
        if table_name not in self.tables:
            self.tables[table_name] = self.client.get_table(self.__table_refs[table_name])

    def table_schema(self, table_name):
        """
        Get the schema for a specific table from a dataset
        """
        self.__fetch_table(table_name)
        return(self.tables[table_name].schema)

    def list_tables(self):
        """
        List the names of the tables in a dataset
        """
        self.__fetch_dataset()
        return([x.table_id for x in self.client.list_dataset_tables(self.dataset)])

    def estimate_query_size(self, query):
        """
        Estimate gigabytes scanned by query.
        Does not consider if there is a cached query table.
        See https://cloud.google.com/bigquery/docs/reference/rest/v2/jobs#configuration.dryRun
        """
        my_job_config = bigquery.job.QueryJobConfig()
        my_job_config.dry_run = True
        my_job = self.client.query(query, job_config=my_job_config)
        return my_job.total_bytes_processed / self.BYTES_PER_GB

    def query_to_pandas(self, query):
        """
        Take a SQL query & return a pandas dataframe
        """
        my_job = self.client.query(query)
        iterator = my_job.result()
        records = [el[0] for el in iterator]
        return pd.DataFrame(records)
        
    def query_to_pandas_safe(self, query, max_gb_scanned=1):
        """
        Execute a query if it's smaller than a certain number of gigabytes
        """
        query_size = self.estimate_query_size(query)
        if query_size <= max_gb_scanned:
            return self.query_to_pandas(query)
        msg = "Query cancelled; estimated size of {0} exceeds limit of {1} GB"
        print(msg.format(query_size, max_gb_scanned))

    def head(self, table_name, num_rows=5, start_index=None, selected_columns=None):
        """
        Get the first n rows of a table as a DataFrame
        """
        self.__fetch_table(table_name)
        active_table = self.tables[table_name]
        schema_subset = None
        if selected_columns:
            schema_subset = [col for col in active_table.schema if col.name in selected_columns]
        results = self.client.list_rows(active_table, selected_fields=schema_subset,
            max_results=num_rows, start_index=start_index)
        results = [x for x in results]
        return pd.DataFrame(
            data=[list(x.values()) for x in results], columns=list(sorted(results[0]._xxx_field_to_index, key=results[0]._xxx_field_to_index.get)))

In [3]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="DataVizMgmt-8bd22854739a.json"#"DataVizMgmt-c847a9a5ec04.json"

nyc = BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="new_york")

In [7]:
uberDataPath = "uberData"
tables = os.listdir(uberDataPath)
bqDfs = []
for t in tables:
    print 'elaborating', t
    df = pd.read_csv(os.path.join(uberDataPath, t))
    dataframeToMongo(df, t)
    

elaborating uber-raw-data-apr14.csv
elaborating uber-raw-data-aug14.csv
elaborating uber-raw-data-janjune-15.csv
elaborating uber-raw-data-jul14.csv
elaborating uber-raw-data-jun14.csv
elaborating uber-raw-data-may14.csv
elaborating uber-raw-data-sep14.csv


In [ ]:
tables = nyc.list_tables()
bqDfs = []
for t in tables:
    if t in ['tlc_yellow_trips_2015'] :
        print 'elaborating', t
        rowsObject = nyc.query_to_pandas("SELECT COUNT(*) FROM `bigquery-public-data.new_york.%s`"% t)
        rows = rowsObject[0][0]
        start = 700000
        size= 100000
        while (size):
            if start + size > rows:
                size = None
                break
            bqDf1 = nyc.head(t, num_rows=size, start_index=start)
            dataframeToMongo(bqDf1, t)
            
            start = start + size
            print 'rows stored:', start

elaborating tlc_yellow_trips_2015
rows stored: 100000
rows stored: 200000
rows stored: 300000
rows stored: 400000
rows stored: 500000
rows stored: 600000


In [ ]:
client = MongoClient('localhost',int(27017))
db = client["DataVizMgmt"]
collection2009 = db['tlc_yellow_trips_2009']
collection2015 = db['tlc_yellow_trips_2015']
taxiRides2009 = collection2009.count()
taxiRides2015 = collection2015.count()
perc = float((taxiRides2015 - taxiRides2009)) / taxiRides2009 * 100
print "Le corse dei taxi sono passate da %d \
    nel 2009 a %d nel 2015 (%f %%)" % (taxiRides2009, taxiRides2015, perc)

In [ ]:
mean2009Cursor = collection2009.aggregate([{"$group": {"_id":None, "amount": {"$avg":"$total_amount"} } }])
mean2015Cursor = collection2015.aggregate([{"$group": {"_id":None, "amount": {"$avg":"$total_amount"} } }])
mean2009 = [el for el in mean2009Cursor][0]['amount']
mean2015 = [el for el in mean2015Cursor][0]['amount']
percMean = (mean2015 - mean2009) / mean2009 * 100
print "La spesa media per le corse taxi è passata dal %f del 2009 al %f nel 2015 (%d %%)" % (mean2009, mean2015, percMean)

In [ ]:
ridesWithTips2009 = collection2009.find({"tip_amount": {"$gt": 0}}).count()
ridesWithTips2015 = collection2015.find({"tip_amount": {"$gt": 0}}).count()
avgTips2009 = float(ridesWithTips2009) / taxiRides2009 * 100
avgTips2015 = float(ridesWithTips2015) / taxiRides2015  * 100
difference = avgTips2009  - avgTips2015
print "Le corse con mancia sono passate dal %f %% del 2009 al %f %% del 2015 (%f %%) " % (avgTips2009, avgTips2015, abs(difference))